In [1]:
import geopandas as gpd

In [2]:
gdb_path = "../data/miami.gdb"

In [3]:
# read layers
fl_blockgroups = gpd.read_file(gdb_path, layer="florida_blockgroups")
acs = gpd.read_file(gdb_path, layer="miami_ACS_blockgroup_2018")
hurricane_evac = gpd.read_file(gdb_path, layer="HurricaneEvac_Zone_proj")

# reproject hurricane_evac to match florida_blockgroups CRS
hurricane_evac = hurricane_evac.to_crs(fl_blockgroups.crs)

/Users/dustinmichels/GitRepos/adv-gis/lab4/.venv/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts.  The processing may be really slow.  You can skip the processing by setting METHOD=SKIP.
  return ogr_read(


In [4]:
# join fl_blockgroups & acs
fl_blockgroups_withACSTable = fl_blockgroups.merge(
    acs,
    left_on="GEOFIPS",
    right_on="Geo_FIPS",
    how="inner"
)

In [5]:
# evac categories
categories = hurricane_evac["CATEGORY"].dropna().unique()
categories

array([1, 2, 3, 0, 4, 5], dtype=int16)

In [6]:
for cat in categories:
    
    # filter rows where CATEGORY == cat
    sel = hurricane_evac[hurricane_evac["CATEGORY"] == cat]

    # intersection
    intersect = gpd.overlay(
        sel,
        fl_blockgroups_withACSTable,
        how="intersection")

    # dissolve
    dissolve = intersect.dissolve(
        by="GEOID",
        aggfunc={
            "SE_A14006_001": "first",
            "SE_A10036_001": "first",
            "SE_A18009_001": "first"
        })

    stats = dissolve.agg({
        "SE_A14006_001": ["median", "mean"],
        "SE_A10036_001": ["median", "mean"]
    })

    # save to CSV
    stats_table = f"out/florida_bg_Category{cat}_statistics.csv"
    stats.to_csv(stats_table)